CO2 Emissions Prediction
Objetivo
Extrair indicadores do World Development Indicators (WDI) via API do Banco Mundial e gerar:
data/raw/wdi_long.csv (formato longo: país–indicador–ano–valor)
data/processed/wdi_wide.csv (formato wide estilo Kaggle: 1 linha por país+indicador, colunas por ano)

In [ ]:
import logging
from pathlib import Path

import requests
import pandas as pd

logging.basicConfig(level=logging.INFO, format="%(asctime)s | %(levelname)s | %(message)s")
logger = logging.getLogger("wdi_etl")

Configurações do ETL
INDICATORS: lista dos códigos WDI (fonte oficial)
YEARS: intervalo 2000–2020
PATH_LONG/PATH_WIDE: saídas em disco

In [1]:
INDICATORS = [
    "EG.ELC.ACCS.ZS",       # Access to electricity (% of population)
    "AG.LND.AGRI.ZS",       # Agricultural land (% of land area)
    "ER.H2O.FWVT.ZS",       # Annual freshwater withdrawals, total (% of internal resources)
    "AG.LND.ARBL.ZS",       # Arable land (% of land area)
    "AG.LND.FRST.ZS",       # Forest area (% of land area)
    "EG.USE.ELEC.KH.PC",    # Electric power consumption (kWh per capita)
    "EG.USE.PCAP.KG.OE",    # Energy use (kg of oil equivalent per capita)
    "EG.ELC.RNEW.ZS",       # Renewable electricity output (% of total electricity output)
    "EG.FEC.RNEW.ZS",       # Renewable energy consumption (% of total final energy consumption)
    "SP.POP.GROW",          # Population growth (annual %)
    "NY.GDP.PCAP.CD",       # GDP per capita (current US$)
    "EN.ATM.CO2E.PC",       # CO2 emissions (metric tons per capita)
]

YEARS = list(range(2000, 2021))
DATE_RANGE = f"{min(YEARS)}:{max(YEARS)}"

BASE = "https://api.worldbank.org/v2"

PATH_LONG = Path("data/raw/wdi_long.csv")
PATH_WIDE = Path("data/processed/wdi_wide.csv")

NameError: name 'Path' is not defined